This script has the following objectives:  
- Reading the data-prod-270222.datascience.counting_baseline_raw table  
- Computing the features on the dataset to classify the rows as IN or OUT  
- Saving the table

The principal aim is to filter out the associated devices and use that as a baseline to add on afterward...

In [7]:
import pandas as pd
import numpy as np
from itertools import chain

#saving bigtable
import pandas_gbq
from datalab.context import Context
import datalab.storage as storage
import datalab.bigquery as bq
from pandas import DataFrame
import time
from tqdm import tqdm
tqdm.pandas()

In [8]:
#Fonctions for feature computation
def transform_column_feature(udf, df, column_to_compute):
    '''
    Transforming a grouped column using a udf, returns a dataframe column
    '''
    new_column = pd.DataFrame(pd.Series(list(chain.from_iterable(df.groupby('device_mac')[column_to_compute].apply(lambda x: udf(x))))))
    return new_column

def rolling_window_column_feature(udf, df, column_to_compute, window_size, new_col_str):
    '''
    Transforming a grouped column using a udf, returns a dataframe
    '''
    start = time.time()
    new_column = (pd.DataFrame(df.groupby('device_mac')[column_to_compute]
                               .rolling(window_size, min_periods=1)
                               .progress_apply(lambda x: udf(x))
                               .reset_index()[column_to_compute]))
    end = time.time()
    print(f"Computed {new_col_str} in: " + str(end - start))
    return new_column
            
def factorize_column(x):
    return pd.factorize(x)[0]
                                  
def len_unique(x):
    return len(np.unique(x))

In [9]:
%%bigquery df_farmboy_matrix
SELECT *
FROM  data-prod-270222.datascience.counting_baseline_raw
limit 100000

Downloading: 100%|██████████| 7972/7972 [00:00<00:00, 11473.72rows/s]


In [10]:
#sorting values needed for correct rolling computation
df_farmboy_matrix['times'] = df_farmboy_matrix['Timestamp']

central_mean = df_farmboy_matrix[df_farmboy_matrix['signal_strength']<-20]['signal_strength'].median()
df_farmboy_matrix['signal_strength'] = df_farmboy_matrix['signal_strength'] - (central_mean)

df_farmboy_matrix.sort_values(['device_mac','Timestamp'], inplace = True)
df_farmboy_matrix.reset_index(inplace=True)

In [11]:
df_farmboy_matrix['times_count_5w'] = transform_column_feature(factorize_column, df_farmboy_matrix, 'times')
df_farmboy_matrix['times_count_5w'] = rolling_window_column_feature(len_unique, df_farmboy_matrix, 'times_count_5w', 5, 'times_count_5w')
df_farmboy_matrix['len_signal_strength'] = rolling_window_column_feature(len, df_farmboy_matrix, 'signal_strength', 1000, 'len_signal_strength')
df_farmboy_matrix['max_signal_strength'] = rolling_window_column_feature(max, df_farmboy_matrix, 'signal_strength', 10, 'max_signal_strength')

7972it [00:00, 19707.96it/s]


Computed times_count_5w in: 0.4100503921508789


7972it [00:00, 38631.63it/s]


Computed len_signal_strength in: 0.2119457721710205


7972it [00:00, 35269.45it/s]

Computed max_signal_strength in: 0.23171567916870117


In [12]:
start = time.time()
df_farmboy_matrix[['device_mac','Timestamp','times_count_5w','signal_strength','len_signal_strength','max_signal_strength']].to_gbq('data-prod-270222.datascience.counting_baseline_matrix', 
                 'data-prod-270222',
                 chunksize=10000, 
                 if_exists='replace'
                 )
end = time.time()
print("time alternative 1 " + str(end - start))

100%|██████████| 1/1 [00:00<00:00, 10082.46it/s]

time alternative 1 3.3416666984558105
